In [7]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [9]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1436, with_units=False)
Hist_table


days,Entries
IntegerAndSexagesimal,Historical
1 ;,"13 ; 10,35"
2 ;,"26 ; 21,10"
3 ;,"01s 09 ; 31,45"
4 ;,"01s 22 ; 42,20"
5 ;,"02s 05 ; 52,55"
6 ;,"02s 19 ; 03,30"
7 ;,"03s 02 ; 14,05"
8 ;,"03s 15 ; 24,40"
9 ;,"03s 28 ; 35,15"


In [10]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(31):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [13]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("13;10,35,1,15,11,4,35"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(31):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=7):
        Test_recomp[i][1]=daily_mean*Hist_table[i][0]%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=60*60*(Hist_table[i][1]-Test_recomp[i][1])


In [14]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(0.0, nan)

In [15]:
B.astype(float).style.background_gradient(axis=None)

,Entries
days,
1 ;,0.000000
2 ;,0.000000
3 ;,0.000000
4 ;,0.000000
5 ;,0.000000
6 ;,0.000000
7 ;,0.000000
8 ;,0.000000
9 ;,0.000000


In [16]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Lunar mean days.csv')
B.to_csv('Lunar mean days.zip', index=False, compression=compression_opts)